In [ ]:
%%time
import tensorflow as tf
import time
import numpy as np
print(tf.config.list_physical_devices('GPU'))

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import BatchNormalization

class dense(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(dense, self).__init__(name='Dense')
        self.num_outputs = num_outputs
        self.bn = BatchNormalization(momentum=0.1)
        
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                     shape = [int(input_shape[-1]),
                                             self.num_outputs])
        
    def call(self, inputs):
        return  self.bn(tf.matmul(inputs, self.kernel))
        

class My_fnn(Model):
    def __init__(self, neurons):
        super().__init__()
        self.activations = []
        self.custom_layers = []
        self.neurons = neurons

        for i, neu in enumerate(neurons):
            layer = dense(neu)
            activation = tf.nn.tanh if i < len(neurons) - 1 else tf.identity
            self.activations.append(activation)
            self.custom_layers.append(layer)
        self.bn = BatchNormalization(momentum=0.1)
    
    def call(self, input_tensor, training=False):
        x = self.bn(input_tensor)
        for i in range(len(self.neurons)):
            x = self.custom_layers[i](x)
            x = self.activations[i](x)
        return x

In [ ]:
model = My_fnn([102,100+2,1])

创建损失函数

In [ ]:
# Define custom loss function
def custom_loss(model, x, y, training):
    y_ = model(x, training=training)
    
    return tf.reduce_mean((y_- y)**2)

In [ ]:
# Define the gradient function
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = custom_loss(model, inputs, targets, training = True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
# optimizer = tf.keras.optimizers.legacy.Adam(1e-3)
# Compile the model
# lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundaries, values=lr_values)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
    
model.compile(optimizer=optimizer, loss=custom_loss)

In [ ]:
def approx_err(model, x, u_reference, mc_rounds):
    l1_err, l2_err, li_err = 0., 0., 0.
    rel_l1_err, rel_l2_err, rel_li_err = 0., 0., 0.
    
    for _ in range(mc_rounds):
        u_approx = model(x, training=False)
        err = tf.abs(u_approx - u_reference)
        
        l1_err += tf.reduce_mean(err)
        l2_err += tf.reduce_mean(err**2)
        li_err = tf.maximum(li_err, tf.reduce_mean(err))
        
        rel_err = err / tf.maximum(u_reference, 1e-8)
        rel_l1_err += tf.reduce_mean(rel_err)
        rel_l2_err += tf.reduce_mean(rel_err**2)
        rel_li_err = tf.maximum(rel_li_err, tf.reduce_max(rel_err))
    
    l1_err /= mc_rounds
    l2_err = np.sqrt(l2_err / mc_rounds)
    rel_l1_err /= mc_rounds
    rel_l2_err = np.sqrt(rel_l2_err / mc_rounds)
    
    return err, l1_err, l2_err, li_err, rel_l1_err, rel_l2_err, rel_li_err

# 训练初始化

In [ ]:
dtype = tf.float32

T, N, d = 1., 1, 100
batch_size = 8192

neurons = [d+100, d+100, 1]
training_steps = 75000

mc_rounds, mc_freq = 10, 100 # Monte Carlo Methods

xi = tf.random.uniform(shape=(batch_size, d), 
                      minval=0.,
                      maxval=1,
                      dtype=dtype)

# u_reference = np.sum(np.sin(xi), axis=1).reshape(-1,1) + 1


def phi(x):
    # Compute the sum of squared elements along axis 1
    result = tf.reduce_sum(x ** 2, axis=1, keepdims=True)
    
    # Cast the result to tf.float32
    return tf.cast(result, tf.float32)

u_reference = phi(xi) + 2.*T*d
# model.summary()

# 训练

In [ ]:
num_epochs = training_steps
file_name = "draft4.txt"
l = []
# Open the file for writing results
with open(file_name, 'w') as file_out:
    file_out.write('step,err,l1_err,l2_err,li_err,l1_rel,l2_rel,li_rel,learning_rate,time_train,time_mc\n')

    for epoch in range(num_epochs):
        start_time = time.time()
    
        loss_value, grads = grad(model, xi, u_reference)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        l.append(loss_value.numpy())

        if (epoch+1)%mc_freq == 0:
            t1_training = time.time()
            err, l1_err, l2_err, li_err, rel_l1_err, rel_l2_err, rel_li_err = approx_err(model, xi, u_reference, mc_rounds)
            end_time = time.time()
            
            current_lr = 1e-3

            # Write results to file
            file_out.write(f'{epoch+1}, {tf.reduce_max(err)}, {l1_err}, {l2_err}, {li_err}, {rel_l1_err}, {rel_l2_err}, {rel_li_err}, {current_lr}, {t1_training - start_time}, {end_time - t1_training}\n')
            file_out.flush()
        
            print(epoch+1, current_lr, loss_value.numpy(), rel_li_err.numpy())

# 可视化 训练结果 误差 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv("draft4.txt")
df.head(10)

In [ ]:
fig2 = plt.figure(dpi = 150, figsize = (10,5))
plt.plot(df["step"], df["l1_rel"], c = 'r', label = "L1 Error")
plt.plot(df["step"], df["l2_rel"], c = 'g', label = "L2 Error")
plt.plot(df["step"], df["li_rel"], c = 'b', label = "Inf Error")
# plt.ylim([0,2])
# plt.xlim([0,3e5])
plt.legend()

# 预测 与 参考值可视化

In [ ]:
# Generate the grid coordinates
xx = np.linspace(0,1,50)
yy = np.linspace(0,1,50)

# Create a meshgrid from the coordinates
X, Y = np.meshgrid(xx, yy)

grid_array = np.column_stack((X.ravel(), Y.ravel()))

In [ ]:
# uu = np.sum(np.sin(grid_array), axis=1) + 1
uu = phi(grid_array) + 2.*T*d

In [ ]:
# Create a 3D plot
fig = plt.figure(figsize=(15,5), dpi=200)
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with color based on function values
scatter = ax.scatter(grid_array[:,0], grid_array[:,1], uu, c=uu, cmap='viridis', marker='+',alpha=0.3)


# Add colorbar to show the corresponding function values
cbar = fig.colorbar(scatter, ax=ax, label='u(1, x, y)')

scatter = ax.scatter(grid_array[:,0], grid_array[:,1], model(grid_array, training=False), alpha=0.2,cmap='viridis', marker='+', c = 'r')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Function Value')
ax.set_title('u(1,x,y)')

plt.show()